## **HW_4**

---

### **Q1**

**概率计算问题**: 给定模型 $\lambda=(A,B,\pi)$, 观察序列 $O=(o_1,o_2,\cdots,o_T)$, 计算在模型 $\lambda$ 下观测序列 $O$ 出现的概率 $P(O|\lambda)$。编写程序向前算法（算法2），计算观测序列的概率。并通过`例10.2`来验证程序是否正确。

> **Ans**
> 
> 定义向前变量 $\alpha_t(i) := P(o_1,o_2,\cdots,o_t,q_t=S_i|\lambda), 1<t<T$, 表示 $t$ 时刻观测序列为 $o_1,o_2,\cdots,o_t$, 且隐藏状态为 $S_i$ 的概率大小, 于是有
> $$
> \begin{aligned}
> \\
> &\alpha_1(i)=\pi_ib_i(o_1),\quad \text{其中}\pi_i\text{表示初始时刻状态为 $i$ 的概率}, \quad b_i(o_t) := P(o_t|q_t=S_i,\lambda), \\
> &\alpha_{t+1}(j) = [\sum_{i=1}^N \alpha_t(i)a_{i,j}] b_j(o_{t+1}),\quad t=1,2,\cdots,T-1,\ 1\le j\le N \\
> \end{aligned}
> $$ 
> 最后有
> $$
> P(O|\lambda) = \sum_{i=1}^N\alpha_T(i)
> $$


In [2]:
import numpy as np

A = np.array([[0.5,0.2,0.3],[0.3,0.5,0.2],[0.2,0.3,0.5]])
B = np.array([[0.5,0.5],[0.4,0.6],[0.7,0.3]])
Pi = np.array([0.2,0.4,0.4])

RED = 0
WHITE = 1
O = [RED,WHITE,RED]

def forwordAlg(transition_matrix=A,emission_matrix=B,init_distribution=Pi,observation_seq=O):
    T = len(observation_seq)
    N = len(transition_matrix)
    # initiation for alpha_1
    Alpha = np.array([init_distribution[i]*emission_matrix[i][observation_seq[0]] for i in range(N)])
    for t in range(1,T):
        Alpha = np.array([Alpha.dot(transition_matrix[:,j])*emission_matrix[j][observation_seq[t]] for j in range(N)])
    print("观测序列为",observation_seq,"的概率为: ",sum(Alpha))
    return sum(Alpha)
print(forwordAlg())


观测序列为 [0, 1, 0] 的概率为:  0.130218
0.130218


### **Q2**

**解码问题**：已知模型 $\lambda=(A,B,\pi)$ 和观测序列 $O=(o_1,o_2,\cdots,o_T)$, 求最有可能对应的状态序列 $Q=(q_1,q_2,\cdots,q_T)$. 即，在该状态序列下，观测序列的条件概率 $P(O|Q)$ 最大。编写Viterbi算法（算法4）并通过`例10.3`验证程序是否正确。

> **Ans**
> 
> 定义目标函数 $\delta_t(i)=\underset{q_1,q_2,\cdots,q_{t-1}}{max} P[q_1q_2\cdots,q_t=S_i,o_1,o_2,\cdots,o_t|\lambda]$, 所以有
> $$
> \begin{aligned}
> \\
> &\delta_{t+1} = \underset{q_1,q_2,\cdots,q_t}{max} P[q_1q_2\cdots,q_t,q_{t+1}=S_i,o_1,o_2,\cdots,o_{t+1}|\lambda] = [\underset{i}{max} \delta_t(i)a_{i,j}]b_j(o_{t+1}) \quad \text{保存路径记为} \quad \Psi_t(j) = \underset{i}{argmax}[\delta_{t-1}a_{ij}] \\
> \end{aligned}
> $$
> 流程如下：
> $$
> \begin{aligned}
> \text{初始时刻}&\\
> &\delta_1(i) = \pi_ib_i(o_1),\,\Psi_1(i) = 0, \quad 1\le i\le N \\
> \text{递推}&\\
> &\delta_t(j) = \underset{1\le i\le N}{\max}[\delta_{t-1}a_{ij}]b_j(o_t),\,\Psi_t(j)=\underset{1\le i\le N}{\argmax}[\delta_{t-1}a_{ij}], \quad 2\le t\le T,\,1\le j\le N \\
> \text{结果}&\\
> &p^*=\underset{1\le i\le N}{\max}[\delta_t(i)],\,q_T^*=\underset{1\le i\le N}{\argmax}[\delta_t(i)] \\
> \text{回溯}&\\
> &q_t^*=\Psi_{t+1}(q_{t+1}^*), \quad t=T-1,T-2,\cdots,1
> \end{aligned}
> $$


In [5]:
import numpy as np
A = np.array([[0.5,0.2,0.3],[0.3,0.5,0.2],[0.2,0.3,0.5]])
B = np.array([[0.5,0.5],[0.4,0.6],[0.7,0.3]])
Pi = np.array([0.2,0.4,0.4])

RED = 0
WHITE = 1
O = [RED,WHITE,RED]

def viterbiAlg(transition_matrix=A,emission_matrix=B,init_distribution=Pi,observation_seq=O):
    T = len(observation_seq)
    N = len(transition_matrix)
    Delta = init_distribution*emission_matrix[:,observation_seq[0]]
    Phi = np.array([ [0 for j in range(N)] for t in range(T)])
    for t in range(1,T):
        tmpDelta = np.array([0.0 for i in range(N)])
        for j in range(N):
            mvlu = np.max(Delta*(transition_matrix[:,j]))
            Phi[t][j] = np.argmax(Delta*(transition_matrix[:,j]))
            tmpDelta[j] = mvlu*emission_matrix[j][observation_seq[t]]
        Delta = tmpDelta.copy()
    Pstar = np.max(Delta)
    Qstar = [np.argmax(Delta)]
    # traceback
    for t in range(T,1,-1):
        tmpQ = Phi[t-1][Qstar[-1]]
        Qstar += [tmpQ]
    print("最有可能状态为: ",Qstar[::-1])
    return Qstar[::-1]

viterbiAlg()

最有可能状态为:  [2, 2, 2]


[2, 2, 2]